In [1]:
import pandas as pd
import json

In [2]:
file_path = 'NER_List_All_18.csv'
df = pd.read_csv(file_path)

# Dictionnaire des remplacements
replacements = {
    'Historias': 'Textos Narrativos',
    'Relaciones': 'Textos Narrativos',
    'Relaciones de sucesos': 'Textos Narrativos',
    'Romances': 'Textos Narrativos',
    'Chascos': 'Textos Narrativos',
    'Canciones': 'Poesías y Canciones',
    'Coplas': 'Poesías y Canciones',
    'Décimas': 'Poesías y Canciones',
    'Glosas': 'Poesías y Canciones',
    'Seguidillas': 'Poesías y Canciones',
    'Trovos': 'Poesías y Canciones',
    'Villancicos': 'Poesías y Canciones',
    'Jácaras': 'Poesías y Canciones',
    'Chistes': 'Poesías y Canciones',
    'Sainetes': 'Textos Dramáticos y Teatrales',
    'Diálogos': 'Textos Dramáticos y Teatrales',
    'Coloquios': 'Textos Dramáticos y Teatrales',
    'Pasillos': 'Textos Dramáticos y Teatrales',
    'Entremeses': 'Textos Dramáticos y Teatrales',
    'Matracas': 'Textos Dramáticos y Teatrales',
    'Cartas': 'Textos Diversos',
    'Evangelios': 'Textos Diversos',
    'Recetas': 'Textos Diversos',
    'Oraciones': 'Textos Diversos',
    'Testamentos': 'Textos Diversos',
    'Sátiras': 'Textos Diversos'
}

# Remplacer les valeurs
df['type_text'] = df['type_text'].replace(replacements)

# Sauvegarder le fichier modifié
output_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/moreno-ner/Ner_All_Typtext.csv'
df.to_csv(output_path, index=False)

print("Les remplacements ont été effectués et le fichier sauvegardé.")

Les remplacements ont été effectués et le fichier sauvegardé.


In [24]:
import pandas as pd

file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Ner_All_Typtext.csv'
df = pd.read_csv(file_path, sep=';', skipinitialspace=True)

# Vérifier les noms de colonnes
print("Noms de colonnes disponibles :")
print(df.columns)

# Afficher les lignes avec des valeurs manquantes ou vides dans `type_text`
missing_type_text = df[df['type_text'].isna() | (df['type_text'] == '')]
print("Lignes avec des valeurs manquantes ou vides dans `type_text` :")
print(missing_type_text)

# Sauvegarder ces lignes dans un fichier CSV pour une analyse plus détaillée
missing_type_text.to_csv('/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/missing_type_text.csv', sep=';', index=False)

# Grouper les données par 'normalized_name' et calculer le type de texte le plus fréquent et son nombre d'occurrences
if 'normalized_name' in df.columns:
    grouped = df.groupby('normalized_name')
    
    def most_frequent_value(series):
        if series.empty:
            return None
        return series.value_counts().idxmax()

    def max_count(series):
        if series.empty:
            return 0
        return series.value_counts().max()

    most_frequent_text_type = grouped['type_text'].agg(most_frequent_value)
    most_frequent_count = grouped['type_text'].agg(max_count)

    # Trouver les identifiants des documents pour les occurrences majoritaires et les formater
    def get_major_ids(group):
        if group['type_text'].empty:
            return ''
        most_frequent = group['type_text'].value_counts().idxmax()
        ids = group[group['type_text'] == most_frequent]['id_doc']
        id_counts = ids.value_counts()
        return ', '.join(f"{id} ({count})" if count > 1 else f"{id}" for id, count in id_counts.items())

    most_frequent_ids = grouped.apply(get_major_ids)

    # Filtrer les résultats pour supprimer les valeurs None ou vides
    results = pd.DataFrame({
        'normalized_name': most_frequent_text_type.index,
        'type_text': most_frequent_text_type.values,
        'occurrence': most_frequent_count.values,
        'id_doc': most_frequent_ids.values
    }).dropna()

    # Extraire les coordonnées (longitude et latitude) sans les grouper
    # Prendre la première instance des coordonnées par 'normalized_name'
    first_instance = df.drop_duplicates(subset=['normalized_name'])
    coordinates = first_instance[['normalized_name', 'longitude', 'latitude']].set_index('normalized_name')

    # Joindre les coordonnées avec le DataFrame des résultats
    results = results.join(coordinates, on='normalized_name')

    # Sauvegarder le nouveau fichier CSV
    output_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/ner_All_typetext_city.csv'
    results.to_csv(output_path, sep=';', index=False)

    print("Le fichier résultat a été créé avec succès.")
else:
    print("La colonne 'normalized_name' n'existe pas dans le fichier CSV.")


Noms de colonnes disponibles :
Index(['index', 'id_doc', 'original_name', 'id_wkd', 'type_place',
       'normalized_name', 'latitude', 'longitude', 'shortTitle', 'pubPlace',
       'printer', 'date', 'type_text', 'genre', 'url'],
      dtype='object')
Lignes avec des valeurs manquantes ou vides dans `type_text` :
Empty DataFrame
Columns: [index, id_doc, original_name, id_wkd, type_place, normalized_name, latitude, longitude, shortTitle, pubPlace, printer, date, type_text, genre, url]
Index: []
Le fichier résultat a été créé avec succès.


In [9]:
# Charger le fichier CSV
file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/ner_Moreno_typetext_city_gutgut.csv'

df = pd.read_csv(file_path)


# Fonction pour créer un GeoJSON point
def create_geojson_point(row):
    geojson_feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['latitude'], row['longitude']]
        },
        "properties": {
            # Ajoutez ici toutes les propriétés supplémentaires que vous souhaitez inclure
            "name": row.get('name', 'No Name')  # Assurez-vous que cette colonne existe ou modifiez en conséquence
        }
    }
    return geojson_feature

# Appliquer la fonction à chaque ligne du DataFrame
df['geojson'] = df.apply(create_geojson_point, axis=1)

KeyError: 'coord'

In [25]:
import pandas as pd

# Spécifiez le chemin de votre fichier CSV
file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/ner_Moreno_typetext_city_gutgut.csv'

# Lire le fichier avec l'encodage incorrect supposé (par exemple 'latin1')
df = pd.read_csv(file_path, sep=',', encoding='latin1', skipinitialspace=True)

# Vérifier les premières lignes pour s'assurer que les données sont correctement lues
print("Aperçu des données :")
print(df.head())

# Sauvegarder le fichier avec l'encodage correct (utf-8)
new_file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/ner_Moreno_typetext_city_UTF8.csv'
df.to_csv(new_file_path, sep=',', encoding='utf-8', index=False)

print(f"Le fichier a été ré-encodé et sauvegardé sous {new_file_path}")


Aperçu des données :
  ï»¿normalized_name;type_text;occurrence;id_doc;longitude;latitude
0  Acebedo;Textos Narrativos;1;Moreno_320;43.0391...               
1  Adra;Textos Narrativos;3;"Moreno_180 (2); More...               
2  Alaejos;Textos Narrativos;2;"Moreno_083; Moren...               
3  Alajerââ¥;Poesââ as y Canciones;1;Moreno...               
4  Alameda;Textos Narrativos;1;Moreno_104;37.2084...               
Le fichier a été ré-encodé et sauvegardé sous /Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/ner_Moreno_typetext_city_UTF8.csv


**Gravure**

In [6]:
import pandas as pd

# Lire le fichier Excel
file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/Woodcuts_all.xlsx'
df = pd.read_excel(file_path)

# Afficher un aperçu des données pour vérifier les colonnes disponibles
print("Aperçu des données :")
print(df.head())

# Liste des catégories à analyser
categories = [
    'personaje_masculino', 'personaje_femenino', 'grupos_personajes', 'muerte', 'religion', 'animales', 'construidos',
    'naturales', 'transporte'
]

# Fonction pour compter les occurrences non-nulles par catégorie pour chaque code pliegos
def count_categories(group):
    result = {}
    for category in categories:
        count = group[category].notna().sum()
        result[category] = count
    return pd.Series(result)

# Grouper par 'Code pliegos' et appliquer la fonction de comptage
grouped = df.groupby('Code pliegos').apply(count_categories).reset_index()

# Joindre les résultats avec le DataFrame original
df = df.drop(columns=categories).drop_duplicates(subset=['Code pliegos'])
df = df.merge(grouped, on='Code pliegos', suffixes=('', '_count'))

# Sauvegarder le nouveau fichier Excel avec les résultats
output_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Index_Grabados_Moreno+Varios_comptes.xlsx'
df.to_excel(output_path, index=False)

print(f"Le fichier résultat a été créé avec succès : {output_path}")


Aperçu des données :
      Code gravure Code pliegos       Page       Coordonnees   Titre pliegos   
0  grabado_b_001_1      BCN_001  BCN_001_1   450,149,950,800  Curiosa xacara  \
1  grabado_b_002_1      BCN_002  BCN_002_1   465,154,400,670    Xacara nueva   
2  grabado_b_002_2      BCN_002  BCN_002_1   950,160,350,650    Xacara nueva   
3  grabado_b_003_1      BCN_003  BCN_003_1  345,256,1200,830   Nuevo romance   
4  grabado_b_004_1      BCN_004  BCN_004_1   921,158,470,735    Blas de Leon   

     Date                 Impresor personaje_masculino personaje_femenino   
0  [s.a.]  Herederos de Juan Jolis                 NaN                NaN  \
1  [s.a.]  Herederos de Juan Jolis              hombre                NaN   
2  [s.a.]  Herederos de Juan Jolis                 NaN              mujer   
3  [s.a.]  Herederos de Juan Jolis                 NaN                NaN   
4  [s.a.]  Herederos de Juan Jolis              hombre                NaN   

  ninos  ...   accion  muerte relig

In [12]:
import pandas as pd

# Lire le fichier Excel avec les comptages
count_file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Index_Grabados_Moreno+Varios_comptes.xlsx'
count_df = pd.read_excel(count_file_path)

# Lire le fichier CSV avec les `id_doc` en essayant de vérifier l'encodage
id_doc_file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/NER_List_All_18.csv'
try:
    id_doc_df = pd.read_csv(id_doc_file_path, encoding='utf-8')
except UnicodeDecodeError:
    id_doc_df = pd.read_csv(id_doc_file_path, encoding='latin1')

# Sélectionner uniquement les colonnes nécessaires pour la jointure
columns_to_merge = [
    'Code pliegos', 'personaje_masculino', 'personaje_femenino',
    'grupos_personajes', 'muerte', 'religion', 'animales', 'construidos', 'naturales', 'transporte'
]
count_df = count_df[columns_to_merge]

# Joindre les deux DataFrames sur `id_doc`
merged_df = id_doc_df.merge(count_df, left_on='id_doc', right_on='Code pliegos', how='left')

# Supprimer la colonne 'Code pliegos' après la jointure si nécessaire
merged_df = merged_df.drop(columns=['Code pliegos'])

# Sauvegarder le nouveau fichier CSV avec les résultats en essayant avec latin1
output_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/moreno-ner/NER_Grabados_Moreno+varios.csv'
merged_df.to_csv(output_path, index=False, encoding='utf-8')


In [14]:
import pandas as pd

# Lire le fichier CSV avec les informations de comptage et les coordonnées
file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/moreno-ner/NER_Grabados_Moreno+varios.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# Liste des catégories à analyser
categories = [
    'personaje_masculino', 'personaje_femenino', 'grupos_personajes', 'muerte', 'religion', 'animales', 'construidos',
    'naturales', 'transporte'
]

# Fonction pour calculer le pourcentage de la catégorie la plus représentée
def calculate_percentage(group):
    total_counts = group[categories].sum()
    max_category = total_counts.idxmax()
    max_count = total_counts.max()
    total = total_counts.sum()
    percentage = (max_count / total) * 100 if total > 0 else 0
    return pd.Series([max_category, percentage], index=['Grabados', 'porcentaje'])

# Fonction pour rassembler les id_doc avec leurs occurrences
def gather_ids(group):
    id_counts = group['id_doc'].value_counts()
    return ', '.join(f"{id} ({count})" if count > 1 else f"{id}" for id, count in id_counts.items())

# Grouper par 'normalized_name' et appliquer les fonctions
grouped = df.groupby('normalized_name').apply(calculate_percentage).reset_index()
grouped_ids = df.groupby('normalized_name').apply(lambda x: pd.Series({
    'id_doc': gather_ids(x),
    'longitude': x['longitude'].iloc[0],
    'latitude': x['latitude'].iloc[0]
})).reset_index()

# Joindre les résultats avec les coordonnées
results = grouped_ids.merge(grouped, on='normalized_name')

# Sauvegarder le nouveau fichier CSV avec les résultats
output_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/GrabadosMoreno+Varios_with_percentages.csv'
results.to_csv(output_path, index=False, encoding='utf-8')

print(f"Le fichier résultat a été créé avec succès : {output_path}")

Le fichier résultat a été créé avec succès : /Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Carto/GrabadosMoreno+Varios_with_percentages.csv
